# Welcome to the Correlation Example!

## Workshop Steps
Now that you have opened up the MyBinder environment and are reading this, you are already on the right track! Inside this environment,
you will also find:
* **sample scripts**: This folder contains the base of the scripts that you will be working with to complete the exercise. Please look for the triple exclamation points (!!!) as that means that you are being asked to write some code to get things to work!
* **README.md**: This is the README file you saw on the Github page containing general information on the scenario.
* **requirements.txt**: A list of the required Python packages (+ their respective versions) that were installed upon startup of MyBinder.
* **start** and **postBuild**: System files containing system configuration details for the MyBinder environment.
* **setup.ipynb**: The file you are reading right now! Think of this as your home page.

### Step 1: Create a Data Job
Based on what we already learnt in the previous scenarios, let's create a new data job. Since this example uses the VDK Control Service, the data job will be automatically registered on the cloud and needs to have a unique name. 

Remember to store the data job in a sub-folder of the home directory ("/home/jovyan") because the Streamlit script (*build_streamlit_dashboard.py*) needs to be outside of the data job folder. 

***Reminder:** You can always run `!vdk --help` to remind yourself of the vdk commands.*

In [ ]:
# Please finish the vdk command for creation of data job
!vdk ...

### Step 2: Work Out the Data Job Template

Now that you have created a data job, please go inside the subfolder and set up the structure of your data job. As in the rest of the examples, we have preliminary prepared the scripts in the "sample scripts" folder, but there are some coding challenges in them for you to play around!

Let's first delete the template files from the data job folder that we will not need by running the below command.

<font color='red'>**TO DO!**</font>
Remember to replace "data-job-name" with the name of your own data job.

In [ ]:
! rm "data-job-name/10_sql_step.sql"
! rm "data-job-name/20_python_step.py"
! rm "data-job-name/README.md"
! rm "data-job-name/requirements.txt"
! rm "data-job-name/config.ini"

Now let's move the sample scripts to the data job subfolder. Please run the code below.

<font color='red'>**TO DO!**</font>
Remember to replace "data-job-name" with the name of your own data job.

In [ ]:
! mv "sample scripts/01_create_covid_cases_usa_daily.sql" ~/data-job-name
! mv "sample scripts/02_create_yankee_candle_reviews.sql" ~/data-job-name
! mv "sample scripts/03_create_yankee_candle_reviews_transformed.sql" ~/data-job-name
! mv "sample scripts/04_create_weekly_correlation.sql"  ~/data-job-name
! mv "sample scripts/10_ingest_covid_data.py" ~/data-job-name
! mv "sample scripts/20_ingest_amazon_reviews.py" ~/data-job-name
! mv "sample scripts/30_transform_amazon_reviews.py" ~/data-job-name
! mv "sample scripts/40_calculate_correlation.py"  ~/data-job-name
! mv "sample scripts/webscrape.py" ~/data-job-name
! mv "sample scripts/config.ini" ~/data-job-name

**Great!** Now you're all set up with the data job:
* You have created a data job.
* You deleted the template files that you do not need.
* You moved the sample scripts we provided to the data job sub-folder.

The next step is to begin working on each script in the data job. Let's get it going!

### Step 3: Data Job - Create the necessary tables in the DB (scripts 01-04.sql)

The first four scripts (.sql) take care of creating the tables that will store the raw and transformed data in the cloud Trino DB. 
The SQL engine would check whether tables with such names already exist in the DB and if not, they will be created, otherwise the step will be skipped.

<font color='red'>**TO DO!**</font> Please open each of the files and make the necessary insertions:
* 01_create_covid_cases_usa_daily.sql - insert a unique table name in line 3
* 02_create_yankee_candle_reviews.sql - insert a unique table name in line 3
* 03_create_yankee_candle_reviews_transformed.sql - insert a unique table name in line 4
* 04_create_weekly_correlation.sql - insert a unique table name in line 8 and the column definitions in line 9 onwards

<font color='green'>**GOOD JOB!**</font>

Congratulations! You've finished with the first part of the data job! 

**Please remember to save the scripts before continuing further.** 

At this point, you could try running the job in order to:
* Check if the first 4 scripts run sucessfully.
* See whether the tables are actually created in the DB.
* Examine the error messages, as the other scripts will throw them as of now.

<font color='red'>**TO DO!**</font>
Remember to replace "data-job-name" with the name of your own data job.

In [ ]:
! vdk run data-job-name

You can now check whether the tables have been created successfully by running the following commands:

In [ ]:
# Load SQL extention
%config SqlMagic.autocommit=False 
%load_ext sql

<font color='red'>**TO DO!**</font>
Remember to replace "name-of-the-table" with the name of the respective table you want to query.

In [ ]:
%sql select * from mysql.default.name-of-the-table

### Step 4: Data Job - Ingest the daily COVID-19 data (10_ingest_covid_data.py)

The aim of this script is to **ingest the cumulative daily COVID-19 data for the US using an [API](https://github.com/M-Media-Group/Covid-19-API)**. This should already be familiar to you from the second example that you already built! The difference here is that we will be extracting US data rather than data for European countries, and we will only need the daily COVID cases. Again, we will use the **incremental ingestion approach**.

As usual, we start with importing libraries and initializing the `run` method of the `job_input` object. We then retrieve any saved data job properties if the job has been run before, or initialize the "last_date_covid" property if there is no such property from before. It will store the last ingested date for this particular table. 

The next step is to **make a GET request** to the COVID API and extract from the returned dictionary only the necessary data - date and number of cases. This information is saved in a dictionary and then transformed into a pandas dataframe. Based on the value of the "last_date_covid" property, we keep only the records which have not been ingested into the table already. If there are any records left in the dataframe after the applied filtering, these are ingested into the respective table (the table created in script "01_create_covid_cases_usa_daily.sql") using the `send_tabular_data_for_ingestion` method of VDK's `job input` object. The last step is to reset the "last_date_covid" property value to the latest date in the COVID source DB table.

<font color='red'>**TO DO!**</font>

We have a couple of tasks for you to complete in this script:
* **line 21** - add VDK's job_input method that is used to get all job properties. The documentation on all VDK's job_input methods available can be found [here](https://github.com/vmware/versatile-data-kit/blob/246008c8fffcac173b6ac3f434814acb6faf16a7/projects/vdk-core/src/vdk/api/job_input.py#L11).
* **line 31** - add the appropriate method of the requests package that makes a GET API request using the URL defined on line 28.
* **line 53** - add name of the table created in script "01_create_covid_cases_usa_daily.sql".

<font color='green'>**GOOD JOB!**</font>

Congratulations! You finished the second part of the data job as well! To check the completed script works as expected, you can run the data job again and look at the status of the various scripts. You could track the log messagesand see how many records were ingested in the first DB table.

In [ ]:
# Remember to replace "data-job-name" with the name of your own data job
! vdk run data-job-name

Let's also check the contents of the table in which we just ingested the COVID-19 records:

In [ ]:
# Remember to replace "name-of-the-table" with the name of the respective table you want to query.
%sql select * from mysql.default.name-of-the-table

<font color='orange'>**ADVICE:**</font>
In case there were issues with running the script that you just filled out and you want to go back and retry, there are a few actions you need to take before rerunning the data job:
* If there were records ingested in the table that stores the daily COVID-19 data, you need to delete it. To do this, execute the following statement: `%sql drop table mysql.default.name-of-the-table` (remember to replace "name-of-the-table" with the name of the respective table you want to delete). Once you rerun the data job, the table will be recreated as per script "01_create_covid_cases_usa_daily.sql"
* You also need to reset the "last_date_covid" job property. This will ensure that once you rerun the data job and the table has been recreated empty,  all records since 2020-01-01 will be reingested again. To do this, in script "10_ingest_covid_data.py" replace lines 21-25 with the following:

```python 
props = job_input.get_all_properties()
props['last_date_covid'] = '2020-01-01'
```


### Step 5: Data Job - Ingest Amazon reviews (20_ingest_amazon_reviews.py)

With the next script things become even more interesting! Here we will scrape the text of all negative Amazon reviews for one of the [most popular Yankee candles](https://www.amazon.com/Yankee-Candle-Large-Balsam-Cedar/dp/B000JDGC78/ref=cm_cr_arp_d_product_top?ie=UTF8) and will ingest them into a table in our Trino DB. In simple terms, webscaping is basically extracting data from web pages. In Python, one of the most popular packages which provides web scraping functionalities is the [BeautifulSoup package](https://pypi.org/project/beautifulsoup4/).

**Let's open the script and walk through what it does.**

We start by performing some already familiar actions - importing packages, initializing the logger, opening up job_input's `run` function and initializing the data job property that we will use to incrementally ingest the Amazon reviews in the respective DB table ("last_date_amazon" property).

<font color='red'>**TO DO!**</font> Use the hints from script "10_ingest_covid_data.py" and initialize the "last_date_amazon" property yourself! (line 28)

<font color='green'>**GOOD JOB!**</font>

Next we initialize a few variables and then enter into a **while loop** (line 37 in the script). The idea of this while loop is to go through the pages with critical Amazon reviews (each page contains 10 reviews, you can check this out on the Amazon website linked above) until the review date reaches back to the last date that is already present in our DB table (as indicated by the "last_date_amazon" job property). In the first iteration of the data job, we will ingest all reviews from 2020-01-01 until today. To iterate over the pages, we use a **parameter** that is inserted into the URL of the web page and is being increased by 1 at the end of each iteration of the while loop (line 77 in the script).

In lines 44-48 of the script we perform **the actual webscraping**. As you can see, we use 3 methods of the webscrape object - `html_code`, `cus_rev` and `rev_date`. Those methods are defined in the file "webscrape.py" that if you remember we moved from the "sample scripts" folder to the data job folder in the beginning of this walkthrough. 

<font color='red'>**TO DO!**</font> Open the "webscrape.py" script and familiarize yourself with the function definitions. How did we manage to link the two scripts and use the functions defined in "webscrape.py" into the "20_ingest_amazon_reviews.py" script? *Hint: look at line 8 in "20_ingest_amazon_reviews.py".*

<font color='green'>**GOOD JOB!**</font>

Since the webscrape object's methods return data only for the current Amazon review page, we need to append the already scraped results into another object that will **store cumulatively all scrapred results** - we've chosen a Python list object to do the job. In lines 50-70, we append the review dates and the actual text reviews into 2 list objects - `rev_result` and `date_result` using for loops. In those for loops, we also perform some cleaning:
* remove reviews with no text - only picture, video or score (lines 52-53 and 60-61).
* transform the review date from the format "Reviewed in the United States on February 14, 2022" to a datetime Python object that looks like "2022-02-14" (lines 63-69).

In lines 73-74, we have another while loop which makes sure that for every review page we scrape, the number of scraped reviews matches the number of review dates or in other words, the length of both lists (`rev_result` and `date_result`) are the same. This step is necessary because in lines 52-53 we skip the reviews containing only images or videos (i.e. review text = "The media could not be loaded."). This means that at each page, we might end up having more review dates than actual review texts. The `date_result.pop(-1)` (line 74) basically removes the last item of the date_result list until the length of `rev_result` and `date_result` are the same. The following example illustrates what the code does in practice:

**Step 1:**

| date_page | rev_page | 
| --- | --- |
| 3/10/2022 | This candle has no scent. |
| 3/11/2022 | The media could not be loaded. |
| 3/12/2022 | The smell used to be much stronger before. |

**Step 2:**

| date_page | rev_page | 
| --- | --- |
| 3/10/2022 | This candle has no scent. |
| 3/11/2022 | The smell used to be much stronger before. |
| 3/12/2022 | 

**Step 3:**

| date_result | rev_result | 
| --- | --- |
| 3/10/2022 | This candle has no scent. |
| 3/11/2022 | The smell used to be much stronger before. |

Of course, this is not the best method that can be chosen since on some pages we might end up with a few days discrepancy between the actual date a given review is written and the date we assign it to in the DB, but since those cases are rare (3-4 such reviews for the entire period since January 2020) and we aim to not overcomplicate the solution, this is a good approximation of the results. 

<font color='red'>**TO DO!**</font> In your spare time, think of a more advanced solution to perfectly match the review dates with the actual text reviews on each page.

After we finish going through the review pages and go out of the while loop, we **zip the two lists into 1 object and convert it to a pandas data frame** (line 80). Since the while loop that goes through the review pages will always execute at least once (current timestamp > last ingested review), the first review page will always be scraped. However, it might contain reviews that were already ingested into the DB. To prevent this, we use a filter that keeps only the non-ingested records using the "last_date_amazon" job property (line 83). 

In lines 84-87 we also perform some **data cleaning** which removes emoji characters from the text reviews. This step is needed since emojis have a non-standard encoding which breaks ingestion into our Trino DB. As you can see, here we also use a method defined in the "webscrape.py" script - `remove_emoji`.

<font color='red'>**TO DO!**</font> Go to the "webscrape.py" script and investigate what the `remove_emoji` method does. Which external package does it use? Google it and find out more about regular expressions and what they do!

We're almost at the end of this script! What is left for us to do is to **ingest** the dataframe values into the DB table we've created in script "02_create_yankee_candle_reviews.sql" and **reset the "last_date_amazon" job property** (lines 90-99).

<font color='red'>**TO DO!**</font> Use your knowledge and hints from the previous script and enter the arguments of the `send_tabular_data_for_ingestion` method by yourself!

One last thing to pay attention to - in line 103 we "pause" the data job execution for 10 seconds. This step is necessary because there is some latency between the execution of the script and the actual ingestion of the data into the DB table. Since the next script that we will be working on (30_transform_amazon_reviews.py) reads from the table that we are now ingesting into, we have to make sure that all needed records will be present in table.

<font color='green'>**GOOD JOB!**</font>

**That was a challenge!**

Let's run the job again and then query the Amazon reviews table just to make sure that the insertions we made in the script are correct. **In case you observe errors in the script that you just editted**, please make sure to repeat the steps we explained in Step 4 of this walkthrough on deleting the Amazon reviews table and resetting the "last_date_amazon" job property.

In [ ]:
# Remember to replace "data-job-name" with the name of your own data job
! vdk run data-job-name

In [ ]:
# Remember to replace "name-of-the-table" with the name of the respective table you want to query.
%sql select * from mysql.default.name-of-the-table

### Step 6: Data Job - Transform the raw Amazon reviews data (30_transform_amazon_reviews.py)

The next milestone in our data job is to read the raw Amazon data we've just ingested and do some transformations. We need to first **flag all reviews containing the words "scent", "smell" or "fragnance"**. This will indicate which of the critical reviews are (potentially) complaints about the candles having no scent. Next, we need to **count how many "no scent" reviews there are per day**.

As in the rest of the scripts, we will be saving data into a new table so we will need a job property to store the last ingested date.

<font color='red'>**TO DO!**</font> 

Your first task is to **complete the job property definition**, this time entirely by yourself! You need to first get all job properties, then check whether a property with the name "last_date_amazon_transformed" already exists in the properties dictionary and if not, initialize it with a value of '2020-01-01'. This definition will be analogous to the the job properties definitions we made in the previous 2 scripts so you can use them as hints. Provide you inputs starting from line 26.

<font color='green'>**GOOD JOB!**</font>

In the next part of the code, we use `job_input`'s `execute_query` method to fetch all records from the table with raw Amazon reviews that we've already populated in the previous script (lines 29-36). Please note that we provide a parameter to the SQL query which is the value of "last_date_amazon_transformed" job property. In this way once the job starts running on a regular basis, we will be selecting only the newly populated reviews that have not been "transformed" yet. **Remember to put the name of the table you populated in script 20_ingest_amazon_reviews.py in line 32.**

<font color='orange'>**NB:**</font> Have you noticed that the `execute_query` method automatically "knew" in which database to look for our table? How cool is that! It's actually pretty easy to setup VDK to work in accordance with any database, you need to just add the configuration settings in the "config.ini" file of the data job. We will investigate this file in more details once we reach the step on data job deployment.

<font color='red'>**TO DO!**</font> Your next task is to transform the result from the query (`reviews_raw`) into a pandas data frame (line 37). To achieve that, you need to [initialize the DataFrame class of the pandas package](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html). 
*Hint: On line 80 in script "20_ingest_amazon_reviews.py" we make a similar definition that you can use as reference, except that instead of the zipped lists, you need to use the `reviews_raw` object.*

<font color='green'>**GOOD JOB!**</font>

Next, we **make the necessary transformations/aggregations** if there is any data returned from the query:

We create a new column in the data frame called 'flag_no_scent' with value TRUE for the cases when the review text contains the words "scent", "fragrance" or "smell" (lines 41-43). Then we use pandas groupby functionality to count the total number of negative reviews (lines 45-47)  and the number of "no scent" reviews per day (49-51). After that we merge the two "sub-dataframes" with the newly calculated columns and fill all missing values with 0 (line 55). The resulting dataframe has 3 columns - date, num_negative_reviews and num_no_scent_reviews, and is ready to be ingested into the table that we created in script "03_create_yankee_candle_reviews_transformed.sql" (**remember to put its name in line 61**).

<font color='orange'>**HOMEWORK:**</font> As you probably already suspect, the way in which we flag the "no scent" reviews has its flaws since there might be negative reviews which criticize something about the candle, but say that it smells good. These cases are rare, but do appear in the data, so our approach to flagging those reviews would produce some "false positive" results. For the purposes of the example and for simplicity, we decided to leave those as they are, but in your spare time you could make a research on more advanced text analytics approaches and try to improve this example. Feel free to experiment!

<font color='red'>**TO DO!**</font>

Your last task in this script would be to update the value of the "last_date_amazon_transformed" job property by taking the maximum value of the 'date' column in the df_group data frame (line 64).

<font color='green'>**GOOD JOB!**</font>

What is left for us before continuing to the last data job script is to run the data job again and observe whether this script will execute successfully.

In [ ]:
# Remember to replace "data-job-name" with the name of your own data job
! vdk run data-job-name

In [ ]:
# Remember to replace "name-of-the-table" with the name of the respective table you want to query.
%sql select * from mysql.default.name-of-the-table

In case of errors in the edits that you just made, remember to delete the table you just ingested into and reset the "last_date_amazon_transformed" job property before rerunning the data job.

### Step 7: Data Job - Make weekly aggregations and calculate correlation coefficients (40_calculate_correlation.py)

In the last script, we **transform the daily COVID and reviews data to weekly** (as of the Monday of each week) and **recalculate the correlation coefficients** for each new week - i.e. as new weekly data comes in, the time series for COVID cases and number of "no scent" reviews are enriched which means that the correlation coefficient as of the given week will change accordingly.

We start in the usual way - by defining a new job property for the correlation table (lines 27-31). We then read the transformed Amazon reviews data and the daily COVID-19 data and transform the results into dataframes (lines 33-49). 

<font color='red'>**TO DO!**</font>

In lines 34 and 43, add the VDK's method that enables executing SQL statements from Python scripts inside data jobs. *Hint: we used it in the previous script as well.* Also, remember to put the correct names of the tables in lines 37 and 46.

<font color='green'>**GOOD JOB!**</font>

In line 52, we merge the two data frames from the previous step and sort by date descending. Then we start with the **transformations**. First, since the number оf COVID-19 cases in the source table are cumulative numbers, we find the new COVID cases diagnosed for the day in lines 57-58. The aggregation of data to weekly is done in lines 60-68 using pandas `resample` method that can make periodic aggregations by performing different calculations - in our case, summing the values. 

Lines 70-76 take care of **calculating the correlation coefficients** and adding them as a column in the data frame. This is done through a for loop that limits the number of observations in the time series and in this way calculates a correlation coefficients as of each week. For example, in the last week of January 2020, the correlation coefficient will be caculated taking into account only the data prior to this date, while the correlation coefficient for last week will take into account all data recorded in the table so far. In this way we would be able to track how the correlation coefficients change over time. 

<font color='red'>**TO DO!**</font>

Line 79 requires your input! Since the 'date' column in the `df_merged_weekly` data frame is in datetime format (i.e. it looks like "2022-02-06T00:00:00"), we want to transform it to look like "2022-02-06". There is a pandas datetime method which handles such conversions - Google it and find out which! *Hint: we used this method in script "20_ingest_amazon_reviews.py".*

<font color='green'>**GOOD JOB!**</font>

We then keep only the records that have not been ingested in the table so far based on the "last_date_correlation" job property (line 82).

In lines 85-95 we ingest the dataframe values into the weekly correlation table and reset the value of the "last_date_correlation" property in case there are any records left after the filtering. **Remember to put the name of the table you created in script 04_create_weekly_correlation.sql in line 90.**

We finished with all the necessary edits! Let's run the job for the last time and check the results.

In [ ]:
# Remember to replace "data-job-name" with the name of your own data job
! vdk run data-job-name

In [ ]:
# Remember to replace "name-of-the-table" with the name of the respective table you want to query.
%sql select * from mysql.default.name-of-the-table

You should now get a success message for every single script, as well as one for the entire data job above them. **If so, congratulations! You have built the entire data job!**

You can actually observe the incremental ingestion effect even now! As we have printed meaningful trace messages into the log, you should see statements like "Success! 0 rows were inserted in ... table." and "No new records to ingest." for the first 3 ingestion scripts. The only ingestion should now happen in the table containing the weekly correlation coefficients.

### Step 8: Schedule the Data Job for Regular Execution and Deploy

Since the correlation analysis that we perform is on a weekly basis, it makes sense to schedule our data job to run once per week. VDK allows the **automatic execution of data jobs by deploying them on a cloud server** which handles the regular execution as per schedule that the user defines. The deployment configurations are entered in the **"config.ini"** file that is required for deployment. If you remember, we already moved a file with such a name from the "sample scripts" folder to the folder of our data job.

Let's open it up and examine the contents.

In the first section [owner], we have specified the **team owning the data job**. In the second section [job] we defined the schedule of execution. It is in cron format (you can use [this website](https://crontab.guru/#*/20_*_*_*_*) to translate the cron schedule into a human-readable form). In this case, we want the schedule to run on the Monday of each week at 00:01am US time. Since VDK uses UTC time for schedule execution, the cron schedule indicates 05:01am UTC time. 

The config file could also include a [contacts] section which specifies whether any **notifications** are sent to specific emails upon job execution success, failure or deployment. In our case, we have left those empty.

The last part of the config file contains the **VDK configuration settings** - the type of DB to which we will be ingesting, the DB location, schema and catalogue. 

For a full list and explanations of the configuration settings you could enter into the "config.ini" file of a data job, you can run the following command:

In [ ]:
!vdk config-help

<font color='red'>**TO DO!**</font> 

Let's now deploy the data job!

Run the command below, but first **remember to replace name-of-data-job with your data job name** after the "-n" and in the directory pathway.

In [ ]:
!vdk deploy -n name-of-data-job -r "Initial deploy" -p /home/jovyan/name-of-data-job

After this is done, follow the prompts in the displayed log to check the deployment status of your data job.

<font color='green'>**GOOD JOB!**</font>

Our DB will now be enriched with new data every week! We are ready to proceed with the dashboard visualizations. 

### Step 9: Build and Run a Streamlit Dashboard

Now that we have finished with the data job, let's build some cool visualizations!

First, we need to **move the streamlit dashboard script** from the sample scripts folder to the main folder.

In [ ]:
! mv "sample scripts/build_streamlit_dashboard.py" ~/

Let's open up the "build_streamlit_dashboard.py" file.

Since you already had quite a lot of work to do in the data job, we have build the streamlit script entirely for you! The **only required input** from you is in line 39 where you have to enter the name of the weekly correlation table that we populated in script "40_calculate_correlation.py". 

Let's go through the contents of the Streamlit dashboard script and see what is done inside. 

After the introductory parts (importing libraries and setting up title and description of the dashboard - line 1-22), we **create a connection to the Trino DB** where our data is stored (lines 25-36). Pay attention that since we will run this script outside of VDK, we cannot use it's `execute_query` method and we need to set up a manual connection using the trino Python package. To get the necessary configurations for the connection, we use environment variables (the statements that look like `os.environ.get(variable_name)`) that were initialized in the "start" system file (located in the main directory) that MyBinder uses to set up the environment in which we are currently executing our scripts. If we are running those scripts locally on our computers (outside of MyBinder), then the contents of the "start" system file should be executed from a terminal before running a data job.

In lines 38-40 we read data from the weekly correlation table using pandas `read_sql_query` method that directly converts the output into a pandas data frame. We transform the date into datetime format (line 42) and then **make a line plot using matplotlib Python package**. The plot will show the number of weekly COVID-19 cases versus "no scent" complaints over time. 

After that we build another line plot that will show **how the correlation coefficients change over time**. In line 66 we first show the current correlation coefficient as a KPI and below we show the line plot (lines 68-71). This time it's plotted using streamlit's built-in method `st.line_chart()`. The last thing we display in the dashboard is a table with the weeks and the respective correaltion coefficients (lines 72-78).

**That's it! Let's now run the Streamlit app and see how the dashboard looks like.**

Run the two cells below one after the other. The first cell will output a link that you can copy-paste into a browser window ONLY AFTER you run the second cell which is the actual activation of the Streamlit application.

If you want to go back to this notebook and make changes to it, you need to kill the Streamlit App by pressing the "Interrupt the Kernel" button (the little square button on the tools ribbon at the top).

In [ ]:
# Output the necessary link - one of the two links links will open up the Streamlit app, the other will show a '403: Forbidden' message
import os
print("Open streamlit (in a new tab) at this link:")
print("https://notebooks.gesis.org/binder/jupyter/user/" + os.environ.get("JUPYTERHUB_USER") + "/proxy/8501/")
print("https://hub.mybinder.turing.ac.uk/user/" + os.environ.get("JUPYTERHUB_USER") + "/proxy/8501/")

In [ ]:
# Activate the Streamlit app. To view the dashboard, open the link produced by the above cell.
!streamlit run build_streamlit_dashboard.py

<font color='red'>**TO DO!**</font>

Let's look through the graphs we plotted and verify whether our assumptions regarding the relationship between COVID cases and "no scent" complaints are correct. 

In the first graph, our expectation is that the lines of number of COVID cases and number of "no scent" complaints should move almost in sync and when there's a peak in the # COVID cases, there should also be an increase in the "no scent" reviews. 

As for the change in correlation coefficients over time shown in the second part of the dashboard, we expect correlation to be weakening recently since the Omicron variant of COVID does not result in loss of scent.

**Are those expectations confirmed by our analysis?**

<font color='green'>**CONGRATULATIONS!**</font>

You reached the end of this example! 